In [1]:
import requests                 # Библиотека работы с HTTP-запросами по API
import json                     # Для обработки полученных результатов запросов
import time                     # Для задержки между запросами
import os                       # Для работы с файлами
import pandas as pd             # Для формирования датафрейма с результатами
import re                       # Для работы с регулярными выражениями
import numpy as np              # Библиотека работы с массивами
import seaborn as sns           # Статистическая визуализация данных
import matplotlib.pyplot as plt # Визуализация данных

In [ ]:
def get_vacancies(job_title, page=0, retries=5):
    '''Функция запроса страницы.
       job_title - наименование вакансии
       page - Индекс страницы
       retries - количество попыток повторения запроса
       (по умолчанию начинается с первой страницы)'''

    # Справочник для переменной GET-запроса
    # Значения для указания значений в справочнике 
    # указаны в документации к API hh.ru по ссылке: 
    # https://github.com/hhru/api
    params = {
        'text': job_title,
        'area': 113,
        'page': page,
        'per_page': 100
    }

    for i in range(retries):
        try:
            req = requests.get('https://api.hh.ru/vacancies', params, timeout=20)
            data = req.content.decode()
            req.close()
            return data
        except requests.exceptions.RequestException as e:
            print(f"Ошибка при запросе страницы {page}: {e}")
            print(f"Попытка {i+1}/{retries}...")
            time.sleep(0.25)

    print(f"Не удалось получить данные для страницы {page}")
    return None

# Сделаем запрос к API hh.ru по вакансиям

vacancies = []
vacancies_ids = set()
job_titles = [
    "data engineer",
    "data analyst",
    "data scientist",
    "аналитик данных",
    "аналитик",
    "analyst",
    "инженер данных",
    "BI",
    "data",
    "дата",
    "ETL",
]

for job_title in job_titles:
    for page in range(0, 100):
        # Запрашиваем страницу с вакансиями
        response = json.loads(get_vacancies(job_title, page))
        # Получаем список вакансий на странице
        items = response['items']
        for item in items:
            # Проверяем, что вакансия не была уже добавлена в список
            if item['id'] not in vacancies_ids:
                # Проверяем, что в названии вакансии есть искомое слово
                if any(title in item['name'].lower() for title in job_titles):
                    # Добавляем вакансию в список
                    vacancies.append(item)
                    # Добавляем id вакансии во множество уже просмотренных id
                    vacancies_ids.add(item['id'])
        # Проверяем, является ли страница последней
        if response['pages'] - page <= 1:
            break
        # Делаем задержку между запросами
        time.sleep(0.25)

# Сохраняем полученные вакансии в файле
with open('data/vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies, f, ensure_ascii=False, indent=4)

print('Вакансии по запросу собраны и сохранены в файл vacancies.json')
# Количество вакансий
print(f"Количество собранных вакансий: {len(vacancies)}")

# Количество уникальных значений ключа 'name'
unique_names = set([v['name'] for v in vacancies])
names_count = len(unique_names)

print(f"Количество уникальных названий вакансий: {names_count}")


In [ ]:
# Теперь получим полные описания для всех собранных вакансий

def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 2 секунды.")
            time.sleep(2)
            continue
    return description

In [ ]:
# Чтение файла vacancies.json и создание словаря vacancies_dict
with open('data/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)
vacancies_dict = {vacancy["id"]: "" for vacancy in vacancies}

# Обход словаря vacancies_dict и заполнение значениями ключа "description"
for vacancy_id in vacancies_dict:
    vacancies_dict[vacancy_id] = get_description(vacancy_id)

# Сохранение результата в файл descriptions.json
with open('data/descriptions.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies_dict, f, ensure_ascii=False, indent=4)

In [ ]:
# Получим незагруженные описания вакансий

# Обход словаря vacancies_dict и заполнение значениями ключа "description"
def fill_vacancy_descriptions(vacancies_dict):
    for vacancy_id in vacancies_dict:
        if not vacancies_dict[vacancy_id]:
            description = get_description(vacancy_id)
            vacancies_dict[vacancy_id] = description
    return vacancies_dict

filled_dict = fill_vacancy_descriptions(vacancies_dict)


In [ ]:
vacancy_id = '79110745' # заменить на нужный номер вакансии

get_description(vacancy_id)
description

In [ ]:
def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data['description']
    else:
        return None

In [ ]:
vacancies


In [ ]:
vacancies_dict

In [ ]:
filled_dict

In [ ]:
total_vacancies = len(filled_dict)
empty_vacancies = sum(1 for desc in filled_dict.values() if not desc)
print(f'total: {total_vacancies}')
print(f'empty: {empty_vacancies}')


In [ ]:
# счетчики в переменной vacancies_dict

total_vacancies = len(vacancies_dict)
empty_vacancies = sum(1 for desc in vacancies_dict.values() if not desc)
filled_vacancies = sum(1 for desc in vacancies_dict.values() if desc)
print(f'total: {total_vacancies}')
print(f'filled: {filled_vacancies}')
print(f'empty: {empty_vacancies}')

In [ ]:
# счетчики в переменной filled_dict

total_vacancies = len(filled_dict)
empty_vacancies = sum(1 for desc in filled_dict.values() if not desc)
filled_in_filled = sum(1 for desc in filled_dict.values() if desc)
print(f'total: {total_vacancies}')
print(f'filled: {filled_in_filled}')
print(f'empty: {empty_vacancies}')

In [ ]:
# счетчики в переменной vacancies

total_in_vacancies = sum(1 for vacancy in vacancies if 'id' in vacancy)
count_empty_description = len([vacancy for vacancy in vacancies if not vacancy['description']])
count_filled_description = len([vacancy for vacancy in vacancies if vacancy['description']])
print(f'total in vacancies: {total_in_vacancies}')
print(f'empty in vacancies: {count_empty_description}')
print(f'filled in vacancies: {count_filled_description}')

In [ ]:
# заполнение пустых описаний в vacancies c vacancies_dict

for vacancy in vacancies:
    if not vacancy['description']:
        description = vacancies_dict.get(str(vacancy['id']), '').strip()
        if description:
            vacancy['description'] = description



In [ ]:
# заполнение пустых описаний в vacancies c filled_dict

for vacancy in vacancies:
    if not vacancy['description']:
        description = filled_dict.get(str(vacancy['id']), '').strip()
        if description:
            vacancy['description'] = description


In [ ]:
vacancies

In [ ]:
def fill_vacancy_descriptions(vacancies_dict, filled_dict=None):
    if filled_dict is None:
        filled_dict = {}
    empty_vacancies = sum([1 for v in vacancies_dict.values() if not v])
    total_vacancies = len(vacancies_dict)
    for i, vacancy_id in enumerate(vacancies_dict):
        try:
            if not vacancies_dict[vacancy_id]:
                description = get_description(vacancy_id)
                vacancies_dict[vacancy_id] = description
                empty_vacancies -= 1
            filled_vacancies = i + 1 - empty_vacancies
            percent_complete = filled_vacancies / total_vacancies * 100
            percent_remaining = empty_vacancies / len(vacancies_dict) * 100
            print(f'Filled {filled_vacancies}/{total_vacancies} '
                  f'({percent_complete:.2f}% complete, {percent_remaining:.2f}% remaining), '
                  f'Current vacancy: {vacancy_id}, '
                  f'Empty vacancies: {empty_vacancies}')
            filled_dict[vacancy_id] = vacancies_dict[vacancy_id]
        except Exception as e:
            print(f'Error occurred at vacancy {vacancy_id}: {e}')
            break
    return filled_dict


In [ ]:
filled_dict = {}

In [ ]:
filled_dict

In [ ]:
filled_dict = fill_vacancy_descriptions(vacancies_dict)
# Здесь можно остановить выполнение функции в любой момент
# и выполнить код ниже, чтобы продолжить выполнение функции


In [ ]:
filled_dict = fill_vacancy_descriptions(vacancies_dict, filled_dict=filled_dict)


In [ ]:
vacancies_dict

In [ ]:
# Далее добавим описания во все наши вакансии

# Добавление ключа "description" для каждой вакансии
for vacancy in vacancies:
    vacancy["description"] = ""

# Запись описания вакансии в ключ "description"
for vacancy in vacancies:
    vacancy_id = vacancy["id"]
    description = vacancies_dict.get(vacancy_id, "Описание вакансии не найдено")
    vacancy["description"] = description

In [ ]:
# Сохраняем полученные вакансии в файле
with open('data/vacancies.json', 'w', encoding='utf-8') as f:
    json.dump(vacancies, f, ensure_ascii=False, indent=4)

In [ ]:
# Проверим, попала ли в выборку вакансия, опубликованная на hh одной из последних:

# Открываем файл с вакансиями и загружаем их в переменную vacancies
with open('data/vacancies.json', 'r', encoding='utf-8') as f:
    vacancies = json.load(f)

# ID вакансии, которую нужно найти
vacancy_id = '78641187'

# Поиск вакансии по ID и вывод ее описания
for vacancy in vacancies:
    if vacancy['id'] == vacancy_id:
        print("Вакансия:\n", vacancy['name'])
        print("\nОбязанности:\n", vacancy['snippet']['responsibility'])
        print("\nТребования:\n", vacancy['snippet']['requirement'])
        break
else:
    print(f"Вакансия с номером {vacancy_id} не найдена")


In [ ]:
# далее загрузим вакансии в пандас датафрейм

# открываем json-файл и загружаем данные
with open('data/vacancies.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# преобразуем json в pandas dataframe
df = pd.json_normalize(data)

In [ ]:
# Сделаем так, чтобы выводились все столбцы датафрейма
pd.set_option('display.max_columns', None)

In [ ]:
df.head(3)


In [ ]:
df['description'].unique()

Видим, что в столбце 'professional_roles' данные не нормализовались. Что бы разобрать вложенный список из professional_roles, применим к столбцу лямбда-функцию, разделим его на два новых столбца, старый удалим:

In [ ]:
df[["professional_roles_id", "professional_roles_name"]] = df[
    "professional_roles"
].apply(
    lambda x: pd.Series(
        {"professional_roles_id": x[0]["id"], "professional_roles_name": x[0]["name"]}
    )
)
df = df.drop("professional_roles", axis=1)


In [ ]:
df.head(3)

In [ ]:
# Выведем названия столбцов
print(df.columns)

In [ ]:
# Проверим, какие столбцы не содержат данные

missing_cols = df.columns[df.isna().all()].tolist()
print(f'Столбцы без данных: {missing_cols}')

In [ ]:
# Избавимся от них

df = df.drop(missing_cols, axis=1)

In [ ]:
df.sample (n= 5 )

In [ ]:
vacancy_id = '79190621' # заменить на нужный номер вакансии

get_description(vacancy_id='79190621')
description

In [ ]:
url = 'https://api.hh.ru/vacancies/79190621'
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
if response.ok:
    data = response.json()
    description = data['description']


In [ ]:
description

In [ ]:
# Избавимся от лишних (неинформативных) столбцов путем невключения их в 
# обновленный датафрейм (apply_alternate_url, url, area.url, type.id, employer.id,
# employer.url, employer.logo_urls, employment.id, employer.logo_urls.original,
# employer.logo_urls.90, employer.logo_urls.240, address.metro_stations, 
# address.id, department.id и другие)
# )

df = df[
    [
        "id",
        "name",
        "snippet.requirement",
        "snippet.responsibility",
        "description",
        "experience.name",
        "published_at",
        "created_at",
        "alternate_url",
        "working_days",
        "working_time_intervals",
        "working_time_modes",
        "accept_temporary",
        "professional_roles_name",
        "accept_incomplete_resumes",
        "department.name",
        "area.name",
        "salary.from",
        "salary.to",
        "salary.currency",
        "salary.gross",
        "type.name",
        "employer.name",
        "employer.alternate_url",
        "employer.vacancies_url",
        "employer.trusted",
        "address.city",
        "address.street",
        "address.building",
        "address.lat",
        "address.lng",
        "address.raw",
        "address.metro.station_name",
        "address.metro.line_name",
        "address.metro.station_id",
        "address.metro.line_id",
        "address.metro.lat",
        "address.metro.lng",
    ]
]

In [ ]:
df.head(3)

In [ ]:
df['employer.trusted'].unique()

<!-- Также видим, что в столбцах snippet.requirement	и snippet.responsibility есть теги. Если в тексте снипета встретилась поисковая фраза (параметр text ), она будет подсвечена тегом highlighttext (из документации по API). Но нам эти теги ни к чему, избавимся от них: -->

In [ ]:
# Функция удаления тегов из текста

# def remove_tags(text):
#     if isinstance(text, str):
#         return re.sub(r'<.*?>', '', text)
#     else:
#         return text

# df[['snippet.requirement', 'snippet.responsibility']] = \
#     df[['snippet.requirement', 'snippet.responsibility']].applymap(remove_tags)

In [ ]:
# # Проверяем результат:

# print(df['snippet.requirement'].iloc[0])
# print(df['snippet.responsibility'].iloc[0])

In [ ]:
# # Далее необходимо очистить датасет от лишних вакансий. Для этого составим облако слов поля professional_roles_name

# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Объединяем все строки в одну
# text_roles = ' '.join(df['professional_roles_name'])

# # Создаем объект WordCloud
# wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(text_roles)

# # Отображаем облако слов
# plt.figure(figsize=(8, 8), facecolor=None)
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.tight_layout(pad=0)
  
# plt.show()


In [ ]:
# # Объединяем все строки в одну
# text_name = ' '.join(df['name'])

# # Создаем объект WordCloud
# wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(text_name)

# # Отображаем облако слов
# plt.figure(figsize=(8, 8), facecolor=None)
# plt.imshow(wordcloud)
# plt.axis('off')
# plt.tight_layout(pad=0)
  
# plt.show()

In [ ]:
df['professional_roles_name'].unique()

In [ ]:
df.loc[df['professional_roles_name'] == 'Финансовый директор (CFO)']

In [ ]:
df.loc[df['professional_roles_name'] .str.contains('по качеству')]

In [ ]:
df.to_csv('data/df.csv', sep=',', encoding='utf-8')

In [ ]:
# Подсчитаем количество каждой ваакансии

value_counts = df['name'].value_counts()
result = pd.DataFrame({'name': value_counts.index, 'count':value_counts.values})
result.sort_values(by='count', ascending=False, inplace=True)
print(result)


In [ ]:
# Посмотрим топ-50 названий вакансий

counts = df['name'].value_counts()
top_50 = counts.head(50)
print(top_50)

In [ ]:
pd.options.display.max_rows = 20

In [ ]:
df.loc[df['name'] .str.contains('УПЗ')]

In [ ]:
# Посмотрим строки датафрейма, название вакансии которых содержат нехарактерные для DA, DE, DS слова

mask = df['name'].str.contains('1С|УПЗ|Сухум')
df.loc[mask, ['id', 'name', 'alternate_url', 'professional_roles_name']]

Избавимся от нерелевантных вакансий, таких как Java Developer, Разработчик C#, Инженер DevOps и даже Инженер-геодезист, непонятно каким образом попавшей в выборку.

In [ ]:
# Удаляем строки из датафрейма по маске
df.drop(df.loc[mask].index, inplace=True)


In [ ]:
# Переиндексируем даатфрейм
df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
# Для каждой вакансии по запросу, включающему ее номер, можно получить полное описание

# ID вакансии, которую нужно получить
vacancy_id = '79110745'

# Справочник для переменной GET-запроса
params = {
    'id': vacancy_id
}

req = requests.get('https://api.hh.ru/vacancies/{}'.format(vacancy_id), params=params, timeout=20)
data = req.content.decode()
req.close()

# Преобразуем ответ сервера в словарь
response = json.loads(data)

# Получаем полное описание вакансии
description = response['description']

In [ ]:
description

In [ ]:
df.loc[df['id'] == '78954091']

In [ ]:
# def get_description_from_api(vacancy_id):
#     """
#     Получает описание вакансии по её id через API hh.ru.
    
#     :param vacancy_id: id вакансии.
#     :return: Описание вакансии.
#     """
#     # Задаем параметры запроса.
#     params = {
#         'host': 'https://api.hh.ru',
#         'user_agent': 'api-test-agent/1.0',
#         'headers': {
#             'User-Agent': 'api-test-agent/1.0'
#         }
#     }
    
#     # Формируем URL запроса.
#     url = f"{params['host']}/vacancies/{vacancy_id}"
    
#     # Отправляем запрос к API.
#     response = requests.get(url, headers=params['headers'])
    
#     # Обрабатываем результат запроса.
#     if response.status_code == 200:
#         vacancy_data = response.json()
#         description = vacancy_data['description']
#         return description
#     else:
#         return None


# def add_description_to_dataframe(df):
#     """
#     Добавляет описание вакансии в датафрейм.
    
#     :param df: Датафрейм с вакансиями.
#     :return: Датафрейм с добавленными описаниями вакансий.
#     """
#     # Создаем новый столбец для описаний вакансий.
#     df['description'] = None
    
#     # Проходим по всем вакансиям в датафрейме.
#     for index, row in df.iterrows():
#         # Получаем id вакансии.
#         vacancy_id = row['id']
        
#         # Получаем описание вакансии.
#         description = get_description_from_api(vacancy_id)
        
#         # Записываем описание вакансии в датафрейм.
#         df.at[index, 'description'] = description
    
#     return df

In [ ]:
# Получаем описания для каждой вакансии датафрейма, записываем результат

def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
            break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description

# Применяем функцию для каждой вакансии в датафрейме и записываем результат в новый столбец
df['description'] = df['id'].apply(get_description)


In [ ]:
# Проверяем, что все значения в столбце id являются строками
print(df['id'].apply(type).value_counts())

# Преобразуем все значения в столбце id в строковый тип
df['id'] = df['id'].astype(str)

# Выбираем только уникальные идентификаторы вакансий
unique_ids = df['id'].unique()
print(len(unique_ids))

# Проверяем, что количество уникальных идентификаторов вакансий совпадает с общим количеством вакансий в датафрейме
print(len(unique_ids) == len(df))


In [ ]:
# Выбираем первые 10 вакансий и выводим их описание
for index, row in df.head(10).iterrows():
    print(get_description(row['id']))


In [ ]:
get_description(79110745)

In [ ]:
def get_description(vacancy_id):
    url = f'https://api.hh.ru/vacancies/{vacancy_id}'
    headers = {'User-Agent': 'Mozilla/5.0'}
    description = ""
    while True:
        try:
            response = requests.get(url, headers=headers)
            if response.ok:
                data = response.json()
                description = data['description']
                break
        except requests.exceptions.RequestException:
            print(f"Ошибка получения описания вакансии {vacancy_id}. Повтор запроса через 5 секунд.")
            time.sleep(5)
            continue
    return description


In [ ]:
df.description.unique()

In [ ]:
# df['description']
df

In [ ]:
get_description_from_api(78298565)

In [ ]:
df['type.name'].unique()

In [ ]:
df.loc[df['type.name'] == 'Рекламная']

In [ ]:
df.to_csv ('data/data.csv', index= False )

In [ ]:
df['description'].isna().sum()

In [ ]:
df = pd.read_csv('data/data.csv')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df = df[
    [
        "id",
        "premium",
        "name",
        "has_test",
        "response_letter_required",
        "published_at",
        "created_at",
        "archived",
        "alternate_url",
        "relations",
        "working_days",
        "working_time_intervals",
        "working_time_modes",
        "accept_temporary",
        "professional_roles_id",
        "professional_roles_name",
        "accept_incomplete_resumes",
        "department.name",
        "area.name",
        "salary.from",
        "salary.to",
        "salary.currency",
        "salary.gross",
        "type.name",
        "employer.name",
        "employer.alternate_url",
        "employer.vacancies_url",
        "employer.trusted",
        "address.city",
        "address.street",
        "address.building",
        "address.lat",
        "address.lng",
        "address.raw",
        "address.metro.station_name",
        "address.metro.line_name",
        "address.metro.station_id",
        "address.metro.line_id",
        "address.metro.lat",
        "address.metro.lng",
    ]
]

In [ ]:
df['response_letter_required'].unique()

In [ ]:
# Напишем функцию, которая переведет все столбцы датасета в numeric
# Если это столбец с датой, переводим в формат datetime

def convert_columns(df):
    for col in df.columns:
        if 'date' in col.lower():
            try:
                df[col] = pd.to_datetime(df[col])
            except:
                pass
        else:
            try:
                df[col] = df[col].astype(float)
            except:
                pass
    return df

In [ ]:
df = convert_columns(df)

In [ ]:
df.info()

In [ ]:
# Уберем теги из описаний вакансий при помощи ранее созданной функции

df[['description']] = df[['description']].applymap(remove_tags)

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f'Количество цифровых признаков -- {len(numeric_cols)}')
print(f'Цифровые признаки: {numeric_cols}')
print(f'Количество строковых признаков -- {len(categorical_cols)}')
print(f'Строковые признаки: {categorical_cols}')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
colours = ['#993366', '#FFFF00']
sns.heatmap(df.isnull(), cmap=sns.color_palette(colours), ax=ax)
# Decorations
plt.title('Матрица пропущенных значений набора данных', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()


In [ ]:
df.describe()


Кластеризация текстовых данных - это сложная задача машинного обучения, но можно использовать различные методы для ее решения. Вот несколько подходов:

1.Кластеризация на основе слов: вы можете использовать методы машинного обучения, такие как K-средних или DBSCAN, чтобы кластеризовать описания вакансий на основе слов в тексте. В этом случае вам нужно будет создать матрицу признаков, используя методы векторизации, такие как TF-IDF или Word2Vec, чтобы преобразовать текст в числовые значения. Затем можно использовать эти числа в качестве входных данных для алгоритма кластеризации.

2.Кластеризация на основе тем: другой подход - использовать тематическое моделирование для кластеризации описаний вакансий на основе тем, которые встречаются в тексте. Например, можно использовать методы LDA (Latent Dirichlet Allocation) или NMF (Non-Negative Matrix Factorization), чтобы найти темы, связанные с определенными профессиональными навыками или областями. Затем можно кластеризовать описания вакансий на основе этих тем.

3.Кластеризация на основе эмбеддингов: еще один подход - использовать предварительно обученные модели эмбеддингов (например, Word2Vec или GloVe), чтобы преобразовать текст в числовые векторы, представляющие значения слов. Затем можно использовать алгоритмы кластеризации, такие как DBSCAN или AgglomerativeClustering, чтобы кластеризовать описания вакансий на основе этих эмбеддингов.

Какой из этих подходов выбрать зависит от вашей задачи и доступных ресурсов. В любом случае, для успешной кластеризации необходимо иметь достаточно большой и разнообразный набор данных.

Воспользуемся Word2Vec:

In [ ]:
import gensim
from gensim.models import Word2Vec

In [ ]:
# Создадим список списков слов для обучения модели Word2Vec. Для этого необходимо разбить каждое предложение 
# на список отдельных слов и добавить каждый список в основной список.

sentences = df["snippet.requirement"].apply(lambda x: x.split() if x else [])

In [ ]:
model = Word2Vec(sentences, vector_size=300, window=5, min_count=1, workers=4)


In [ ]:
vectors = df['snippet.requirement'].apply(lambda x: model.wv[x.split()]).tolist()

In [ ]:
df['snippet.requirement'].isna().sum()

In [ ]:
df.shape

In [ ]:
df = df.dropna(subset=['snippet.requirement'])

In [ ]:
df.shape

In [ ]:
import numpy as np
vectors = np.array(df['snippet.requirement'].apply(lambda x: model.wv[x.split()]).tolist())


Для кластеризации векторов описаний вакансий можно использовать алгоритм кластеризации KMeans, который является одним из наиболее распространенных методов машинного обучения для кластеризации данных.

In [ ]:
from sklearn.cluster import KMeans

# Выберем количество кластеров
num_clusters = 10

# Инициализируем kmeans
kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=100)

# Обучаем kmeans на наших векторах
X = vectors
kmeans.fit(X)

# Получаем метки кластеров для наших векторов
cluster_labels = kmeans.labels_


In [ ]:
# Ошибка выше происходит из-за того, что массив содержит векторы разной длины
# Определим максимальную длину вектора среди всех векторов в списке:
max_len = max(len(v) for v in vectors)

# Добавим недостающие элементы в конец каждого вектора, заполнив их нулями:
for i, v in enumerate(vectors):
    vectors[i] = np.pad(v, (0, max_len - len(v)), 'constant')